In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd

import time
import os.path
import MapMatching4GMNS

MapMatching4GMNS, version 0.2.14


In [2]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

## step 1 Convert utd Data into GMNS Format

In [3]:
"Convert utd Data into GMNS Format"
files= os.listdir(os.getcwd()) 
for file in files: 
    if file[:5] == 'links':
        utd = pd.read_csv(file)
    if file[:9] == 'detectors':
        detectors = pd.read_csv(file)
        break
utd.rename(columns = {'Unnamed: 0':'utd','long':'utd_long','lat':'utd_lat','citycode':'utd_citycode'}, inplace = True)
detectors.rename(columns = {'Unnamed: 0':'detectors','long':'detectors_long',\
                'lat':'detectors_lat','citycode':'detectors_citycode','length':'detectors_length'}, inplace = True)
detectors['linkid'] = detectors['linkid'].astype(int)
detectors['lanes'] = detectors['lanes'].astype(int)

detectors = detectors.drop(detectors[detectors['detid']=='N29121Z'].index) #dupicate and error


utd_detectors = utd.merge(detectors, on='linkid', how='left')  #miss one detector 89950

In [4]:
utd.to_csv('utd.csv')
utd

,utd,utd_long,utd_lat,order,piece,linkid,group,utd_citycode
0,89398,-2.264742,53.485628,1,1,0,0.1,manchester
1,89399,-2.264694,53.486187,2,1,0,0.1,manchester
2,89400,-2.264653,53.486820,3,1,0,0.1,manchester
3,89401,-2.264374,53.487190,4,1,0,0.1,manchester
4,89402,-2.263860,53.487496,5,1,0,0.1,manchester
...,...,...,...,...,...,...,...,...
937,90335,-2.252004,53.481483,1,1,180,180.1,manchester
938,90336,-2.252378,53.481538,2,1,180,180.1,manchester
939,90337,-2.252817,53.481670,3,1,180,180.1,manchester
940,90338,-2.253200,53.481823,4,1,180,180.1,manchester


In [5]:
detectors

,detectors,detid,detectors_length,pos,fclass,road,limit,detectors_citycode,lanes,linkid,detectors_long,detectors_lat
0,15204,N06111V,0.119161,0.116664,trunk,Great Ancoats Street,48,manchester,2,72,-2.230164,53.483429
1,15205,N06112W,0.084072,0.076377,trunk,Great Ancoats Street,48,manchester,3,76,-2.228188,53.482206
2,15206,N06141A,0.417602,0.414577,trunk,Oldham Road,48,manchester,2,48,-2.224160,53.488127
3,15207,N06141C,0.114765,0.112883,trunk,Oldham Road,48,manchester,2,44,-2.231587,53.485518
4,15208,N06141Y,0.256407,0.249730,other,Thompson Street,0,manchester,1,42,-2.232718,53.487730
...,...,...,...,...,...,...,...,...,...,...,...,...
176,15380,N06482F,0.243023,0.229623,trunk,Chapel Street,32,manchester,2,127,-2.259198,53.483305
177,15381,N06482K,0.035744,0.020701,other,St Philip's Place,0,manchester,2,129,-2.262977,53.483618
178,15382,N06482J,0.107132,0.103730,trunk,Chapel Street,32,manchester,2,130,-2.264217,53.483308
179,15383,N06551T,0.446633,0.369536,trunk,Crescent,32,manchester,2,132,-2.265183,53.483210


In [7]:
print(detectors[detectors['linkid'].duplicated()])

Empty DataFrame
Columns: [detectors, detid, detectors_length, pos, fclass, road, limit, detectors_citycode, lanes, linkid, detectors_long, detectors_lat]
Index: []


In [8]:
utd_detectors.to_csv('utd_detectors.csv')
utd_detectors

,utd,utd_long,utd_lat,order,piece,linkid,group,utd_citycode,detectors,detid,detectors_length,pos,fclass,road,limit,detectors_citycode,lanes,detectors_long,detectors_lat
0,89398,-2.264742,53.485628,1,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
1,89399,-2.264694,53.486187,2,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
2,89400,-2.264653,53.486820,3,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
3,89401,-2.264374,53.487190,4,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
4,89402,-2.263860,53.487496,5,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,90335,-2.252004,53.481483,1,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
938,90336,-2.252378,53.481538,2,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
939,90337,-2.252817,53.481670,3,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
940,90338,-2.253200,53.481823,4,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538


In [31]:
num_link = 0
'''build node.csv'''
print('converting utd data into gmns format...')


node_utd = pd.DataFrame()
node_utd['name'] = None
node_utd['x_coord'] = None
node_utd['y_coord'] = None
node_utd['z_coord'] = None
node_utd['node_type'] = None
node_utd['ctrl_type'] = None
node_utd['zone_id'] = None
node_utd['parent_node_id'] = None
node_utd['geometry'] = None

for i in range(len(utd)-1):
    if utd.loc[i+1,'order'] > utd.loc[i,'order']:
        node_utd = node_utd.append({'name': utd.loc[i,'utd'],\
                                    'x_coord': utd.loc[i,'utd_long'], \
                                    'y_coord': utd.loc[i,'utd_lat'],\
                                    'z_coord': None,\
                                    'node_type': utd.loc[i,'linkid'],\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + utd.loc[i,'utd_long'].astype(str) + " " + utd.loc[i,'utd_lat'].astype(str) +")"}, ignore_index=True)
        num_link += 1
    else:
        
        node_utd = node_utd.append({'name': utd.loc[i,'utd'],\
                                    'x_coord': utd.loc[i,'utd_long'], \
                                    'y_coord': utd.loc[i,'utd_lat'],\
                                    'z_coord': None,\
                                    'node_type': utd.loc[i,'linkid'],\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + utd.loc[i,'utd_long'].astype(str) + " " + utd.loc[i,'utd_lat'].astype(str) +")"}, ignore_index=True)

node_utd = node_utd.append({'name': utd.loc[i+1,'utd'],\
                                    'x_coord': utd.loc[i+1,'utd_long'], \
                                    'y_coord': utd.loc[i+1,'utd_lat'],\
                                    'z_coord': None,\
                                    'node_type': utd.loc[i+1,'linkid'],\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + utd.loc[i+1,'utd_long'].astype(str) + " " + utd.loc[i+1,'utd_lat'].astype(str) +")"}, ignore_index=True)


node_utd.index.name = 'node_id'
node_utd.index += 100000001 #index from 0



node_utd.to_csv('node_utd.csv')
print('node_utd.csv generated!')

converting utd data into gmns format...
node_utd.csv generated!


In [32]:
num_link

761

In [11]:
node_utd

,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,parent_node_id,geometry
node_id,,,,,,,,,
100000001,89398,-2.264742,53.485628,None,0,None,None,None,POINT (-2.264742 53.48562800000001)
100000002,89399,-2.264694,53.486187,None,0,None,None,None,POINT (-2.264694 53.486187)
100000003,89400,-2.264653,53.486820,None,0,None,None,None,POINT (-2.264653 53.48682)
100000004,89401,-2.264374,53.487190,None,0,None,None,None,POINT (-2.264374 53.48719000000001)
100000005,89402,-2.263860,53.487496,None,0,None,None,None,POINT (-2.26386 53.48749599999999)
...,...,...,...,...,...,...,...,...,...
100000938,90335,-2.252004,53.481483,None,180,None,None,None,POINT (-2.2520041 53.48148270000001)
100000939,90336,-2.252378,53.481538,None,180,None,None,None,POINT (-2.252378 53.481538)
100000940,90337,-2.252817,53.481670,None,180,None,None,None,POINT (-2.2528169 53.4816695)


In [12]:
utd_detectors

,utd,utd_long,utd_lat,order,piece,linkid,group,utd_citycode,detectors,detid,detectors_length,pos,fclass,road,limit,detectors_citycode,lanes,detectors_long,detectors_lat
0,89398,-2.264742,53.485628,1,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
1,89399,-2.264694,53.486187,2,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
2,89400,-2.264653,53.486820,3,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
3,89401,-2.264374,53.487190,4,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
4,89402,-2.263860,53.487496,5,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,90335,-2.252004,53.481483,1,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
938,90336,-2.252378,53.481538,2,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
939,90337,-2.252817,53.481670,3,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
940,90338,-2.253200,53.481823,4,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538


In [13]:
'''build link_utd.csv'''
link_utd = pd.DataFrame()
link_utd['name'] = None
link_utd['corridor_id'] = None
link_utd['corridor_link_order'] = None
link_utd['from_node_id'] = None
link_utd['to_node_id'] = None
link_utd['directed'] = None
link_utd['geometry_id'] = None
link_utd['geometry'] = None
link_utd['dir_flag'] = None
link_utd['parent_link_id'] = None
link_utd['length'] = None
link_utd['grade'] = None
link_utd['facility_type'] = None
link_utd['capacity'] = None
link_utd['free_speed'] = None
link_utd['lanes'] = None
link_utd['detid'] = None

for i in range(0,len(utd_detectors)-1):
    if utd_detectors.loc[i+1,'order'] > utd_detectors.loc[i,'order']:
        link_utd = link_utd.append({'name': utd_detectors.loc[i,'utd'],\
                                    'corridor_id': utd_detectors.loc[i,'road'],\
                                    'corridor_link_order' : utd_detectors.loc[i,'order'],\
                                    'from_node_id': node_utd[(node_utd['x_coord']==utd_detectors.loc[i,'utd_long']) & \
                                                    (node_utd['y_coord']==utd_detectors.loc[i,'utd_lat'])].index.values[0], \
                                    'to_node_id': node_utd[(node_utd['x_coord']==utd_detectors.loc[i+1,'utd_long']) & \
                                                    (node_utd['y_coord']==utd_detectors.loc[i+1,'utd_lat'])].index.values[0],\
                                    'directed': None,\
                                    'geometry_id': None,\
                                    'geometry': "LINESTRING (" + utd_detectors.loc[i,'utd_long'].astype(str) + " " + \
                                                utd_detectors.loc[i,'utd_lat'].astype(str) + "," + \
                                                utd_detectors.loc[i+1,'utd_long'].astype(str) +" "+ \
                                                utd_detectors.loc[i+1,'utd_lat'].astype(str) + ")",\
                                    'dir_flag': None,\
                                    'parent_link_id': None,\
                                    'length': None,\
                                    'grade': utd_detectors.loc[i,'fclass'],\
                                    'facility_type': None ,\
                                    'capacity':None,\
                                    'free_speed':utd_detectors.loc[i,'limit'],\
                                    'lanes': utd_detectors.loc[i,'lanes'],\
                                    'detid': utd_detectors.loc[i,'detid']}, ignore_index=True)
link_utd.index.name = 'link_id'
link_utd.index += 100000001


link_utd.to_csv('link_utd.csv')
print('link_utd.csv generated!')


link_utd.csv generated!


In [14]:
link_utd

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,parent_link_id,length,grade,facility_type,capacity,free_speed,lanes,detid
link_id,,,,,,,,,,,,,,,,,
100000001,89398,Silk Street,1,100000001,100000002,None,None,"LINESTRING (-2.264742 53.48562800000001,-2.264...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000002,89399,Silk Street,2,100000002,100000003,None,None,"LINESTRING (-2.264694 53.486187,-2.264653 53.4...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000003,89400,Silk Street,3,100000003,100000004,None,None,"LINESTRING (-2.264653 53.48682,-2.264374 53.48...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000004,89401,Silk Street,4,100000004,100000005,None,None,"LINESTRING (-2.264374 53.48719000000001,-2.263...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000005,89402,Silk Street,5,100000005,100000006,None,None,"LINESTRING (-2.26386 53.48749599999999,-2.2628...",None,None,None,primary,None,None,48.0,1.0,N31111S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000757,90333,Old Mill Street,2,100000936,100000937,None,None,"LINESTRING (-2.223232 53.48135500000001,-2.224...",None,None,None,tertiary,None,None,0.0,1.0,N06271H
100000758,90335,Leftbank,1,100000938,100000939,None,None,"LINESTRING (-2.2520041 53.48148270000001,-2.25...",None,None,None,other,None,None,48.0,1.0,N06521G
100000759,90336,Leftbank,2,100000939,100000940,None,None,"LINESTRING (-2.252378 53.481538,-2.2528169 53....",None,None,None,other,None,None,48.0,1.0,N06521G


In [16]:
'''build measurement_utd.csv''' 

files= os.listdir(os.getcwd()) 
for file in files: 
    if file[:4] == 'data':
        reading = pd.read_csv(file)
        break

# reading = reading[pd.to_datetime(reading['measurement_tstamp'], format='%Y-%m-%d %H:%M:%S')<datetime.datetime.strptime('2015-04-01 02:00:00', '%Y-%m-%d %H:%M:%S')]
reading = reading.loc[0:1500]

measurement_utd = pd.DataFrame()
measurement_utd['name'] = None
measurement_utd['corridor_id'] = None
measurement_utd['corridor_link_order'] = None
measurement_utd['from_node_id'] = None
measurement_utd['to_node_id'] = None
measurement_utd['timestamp'] = None
measurement_utd['volume'] = None
measurement_utd['travel_time'] = None
measurement_utd['speed'] = None
measurement_utd['reference_speed'] = None
measurement_utd['density'] = None
measurement_utd['queue'] = None
measurement_utd['notes'] = None



reading_dict = {}
gp = reading.groupby('detid')
for key, form in gp:
    reading_dict[key] = {
        'day':form['day'].tolist(),
        'interval':form['interval'].tolist(),
        'flow':form['flow'].tolist(),
        'occ':form['occ'].tolist(),
        'speed':form['speed'].tolist()
        }



for i in link_utd.index:
    try:
        reading_dict_selected = reading_dict[link_utd['detid'][i]]
        for j in range(len(reading_dict_selected['day'])):
            measurement_utd = measurement_utd.append({'name': link_utd['name'][i],\
                                            'corridor_id': link_utd['corridor_id'][i],\
                                            'corridor_link_order' : link_utd['corridor_link_order'][i],\
                                            'from_node_id': link_utd.loc[i,'from_node_id'], \
                                            'to_node_id': link_utd.loc[i,'to_node_id'], \
                                            'timestamp': reading_dict_selected['day'][j]+'T'+str(divmod(reading_dict_selected['interval'][j], 3600)[0])+':'\
                                                +str(int(divmod(reading_dict_selected['interval'][j], 3600)[1]/60)),\
                                            'volume': reading_dict_selected['flow'][j],\
                                            'travel_time': None,\
                                            'speed': reading_dict_selected['speed'][j],\
                                            'reference_speed': None,\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }, ignore_index=True)
    except:
        measurement_utd = measurement_utd.append({'name': link_utd['name'][i],\
                                        'corridor_id': link_utd['corridor_id'][i],\
                                        'corridor_link_order' : link_utd['corridor_link_order'][i],\
                                        'from_node_id': link_utd.loc[i,'from_node_id'], \
                                        'to_node_id': link_utd.loc[i,'to_node_id'], \
                                        'timestamp': None,\
                                        'volume': None,\
                                        'travel_time': None,\
                                        'speed': None,\
                                        'reference_speed': None,\
                                        'density': None,\
                                        'queue': None,\
                                        'notes': None }, ignore_index=True)

measurement_utd.to_csv('measurement_utd.csv',index = False)
print('measurement_utd.csv generated!')

measurement_utd.csv generated!


In [17]:
measurement_utd

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,timestamp,volume,travel_time,speed,reference_speed,density,queue,notes
0,89398,Silk Street,1,100000001,100000002,2017-09-29T23:10,24.0,None,40.000000,None,None,None,None
1,89398,Silk Street,1,100000001,100000002,2017-09-29T23:15,96.0,None,41.220339,None,None,None,None
2,89398,Silk Street,1,100000001,100000002,2017-09-29T23:20,120.0,None,38.000000,None,None,None,None
3,89398,Silk Street,1,100000001,100000002,2017-09-29T23:25,72.0,None,35.945946,None,None,None,None
4,89398,Silk Street,1,100000001,100000002,2017-09-29T23:30,96.0,None,35.000000,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6259,90338,Leftbank,4,100000941,100000942,2017-09-29T23:25,162.0,None,37.000000,None,None,None,None
6260,90338,Leftbank,4,100000941,100000942,2017-09-29T23:30,144.0,None,37.000000,None,None,None,None
6261,90338,Leftbank,4,100000941,100000942,2017-09-29T23:35,210.0,None,30.712980,None,None,None,None
6262,90338,Leftbank,4,100000941,100000942,2017-09-29T23:40,264.0,None,30.000000,None,None,None,None


## step 2 map matching

In [18]:
utd_detectors

,utd,utd_long,utd_lat,order,piece,linkid,group,utd_citycode,detectors,detid,detectors_length,pos,fclass,road,limit,detectors_citycode,lanes,detectors_long,detectors_lat
0,89398,-2.264742,53.485628,1,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
1,89399,-2.264694,53.486187,2,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
2,89400,-2.264653,53.486820,3,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
3,89401,-2.264374,53.487190,4,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
4,89402,-2.263860,53.487496,5,1,0,0.1,manchester,15351.0,N31111S,0.620643,0.100084,primary,Silk Street,48.0,manchester,1.0,-2.260317,53.489087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,90335,-2.252004,53.481483,1,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
938,90336,-2.252378,53.481538,2,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
939,90337,-2.252817,53.481670,3,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538
940,90338,-2.253200,53.481823,4,1,180,180.1,manchester,15309.0,N06521G,0.112340,0.086766,other,Leftbank,48.0,manchester,1.0,-2.252378,53.481538


In [19]:
for i in range(0,len(utd_detectors)-1):
    if utd_detectors.loc[i+1,'order'] > utd_detectors.loc[i,'order']:
        link_utd = link_utd.append({'name': utd_detectors.loc[i,'utd'],\
                                    'corridor_id': utd_detectors.loc[i,'road'],\
                                    'corridor_link_order' : utd_detectors.loc[i,'order'],\
                                    'from_node_id': node_utd[(node_utd['x_coord']==utd_detectors.loc[i,'utd_long']) & \
                                                    (node_utd['y_coord']==utd_detectors.loc[i,'utd_lat'])].index.values[0], \
                                    'to_node_id': node_utd[(node_utd['x_coord']==utd_detectors.loc[i+1,'utd_long']) & \
                                                    (node_utd['y_coord']==utd_detectors.loc[i+1,'utd_lat'])].index.values[0],\
                                    'directed': None,\
                                    'geometry_id': None,\
                                    'geometry': "LINESTRING (" + utd_detectors.loc[i,'utd_long'].astype(str) + " " + \
                                                utd_detectors.loc[i,'utd_lat'].astype(str) + "," + \
                                                utd_detectors.loc[i+1,'utd_long'].astype(str) +" "+ \
                                                utd_detectors.loc[i+1,'utd_lat'].astype(str) + ")",\
                                    'dir_flag': None,\
                                    'parent_link_id': None,\
                                    'length': None,\
                                    'grade': utd_detectors.loc[i,'fclass'],\
                                    'facility_type': None ,\
                                    'capacity':None,\
                                    'free_speed':utd_detectors.loc[i,'limit'],\
                                    'lanes': utd_detectors.loc[i,'lanes'],\
                                    'detid': utd_detectors.loc[i,'detid']}, ignore_index=True)

In [20]:
'''build trace.csv'''
'''trace_id is numeric'''
trace = pd.DataFrame()
trace['corridor_id'] = None
trace['agent_id'] = None
trace['date'] = None
trace['utd'] = None
trace['trace_id'] = None
trace['hh'] = None
trace['mm'] = None
trace['ss'] = None
trace['y_coord'] = None
trace['x_coord'] = None

agent_id = 1
trace_id = 0

for i in range(0,len(utd_detectors)-1):
    if utd_detectors.loc[i+1,'order'] > utd_detectors.loc[i,'order']:
        trace = trace.append({'corridor_id': utd_detectors.loc[i,'road'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'utd': utd_detectors.loc[i,'utd'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': utd_detectors.loc[i,'utd_lat'],\
                                    'x_coord': utd_detectors.loc[i,'utd_long']}, ignore_index=True)
        trace_id += 1
    else:
        trace = trace.append({'corridor_id': utd_detectors.loc[i,'road'],\
                                    'agent_id': agent_id,\
                                    'date': None, \
                                    'utd': utd_detectors.loc[i,'utd'],\
                                    'trace_id': trace_id,\
                                    'hh': None,\
                                    'mm': 'None',\
                                    'ss': None,\
                                    'y_coord': utd_detectors.loc[i,'utd_lat'],\
                                    'x_coord': utd_detectors.loc[i,'utd_long']}, ignore_index=True)

        agent_id += 1
        trace_id = 0
        
trace = trace.append({'corridor_id': utd_detectors.loc[i,'road'],\
                            'agent_id': agent_id,\
                            'date': None, \
                            'utd': utd_detectors.loc[i,'utd'],\
                            'trace_id': trace_id,\
                            'hh': None,\
                            'mm': 'None',\
                            'ss': None,\
                            'y_coord': utd_detectors.loc[i,'utd_lat'],\
                            'x_coord': utd_detectors.loc[i,'utd_long']}, ignore_index=True)

trace.to_csv('trace.csv',index = False)
print('trace.csv generated!')

trace.csv generated!


In [21]:
trace

,corridor_id,agent_id,date,utd,trace_id,hh,mm,ss,y_coord,x_coord
0,Silk Street,1,None,89398,0,None,None,None,53.485628,-2.264742
1,Silk Street,1,None,89399,1,None,None,None,53.486187,-2.264694
2,Silk Street,1,None,89400,2,None,None,None,53.486820,-2.264653
3,Silk Street,1,None,89401,3,None,None,None,53.487190,-2.264374
4,Silk Street,1,None,89402,4,None,None,None,53.487496,-2.263860
...,...,...,...,...,...,...,...,...,...,...
937,Leftbank,181,None,90335,0,None,None,None,53.481483,-2.252004
938,Leftbank,181,None,90336,1,None,None,None,53.481538,-2.252378
939,Leftbank,181,None,90337,2,None,None,None,53.481670,-2.252817
940,Leftbank,181,None,90338,3,None,None,None,53.481823,-2.253200


In [12]:
"Get the OSM Network"
network_type = 'auto'
import osm2gmns as og
files= os.listdir(os.getcwd()) 
for file in files: 
    if file[-4:] == '.osm':
        net = og.getNetFromOSMFile(file,network_type=(network_type), default_lanes=True, default_speed=True)
        break

og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=os.getcwd())

osm2gmns, version 0.4.2


In [22]:
# """install MapMatching4GMNS"""
# !pip install MapMatching4GMNS --upgrade


if __name__ == "__main__":
    #If the output data exists, please remove.
    if(os.path.exists("agent.csv")):
       os.remove("agent.csv")
    if(os.path.exists("route.csv")):
       os.remove("route.csv")
    if(os.path.exists("link_performance.csv")):
       os.remove("link_performance.csv")   
    
    start = time.time()
    MapMatching4GMNS.map_match() #input:link.csv,node.csv,trace.csv output:route.csv
    end = time.time()
    
    print('MapMatching4GMNS time cost: %.6f seconds' % (end - start))
    print("The output data is generated!")

call MapMatching4GMNS  dynamic library
\MapMatching4GMNS run starts
\MapMatching4GMNS run completes
MapMatching4GMNS time cost: 0.904013 seconds
The output data is generated!


## step 3 link performance of osm

In [23]:
link_road = pd.read_csv('link.csv', low_memory=False)
trace = pd.read_csv('trace.csv', low_memory=False,index_col=None)
measurement_utd = pd.read_csv('measurement_utd.csv',low_memory=False,index_col=None)
mapping = pd.read_csv('route.csv', low_memory=False,index_col=None)


In [24]:
mapping

,agent_id,from_node_id,to_node_id,timestamp,cumu_distance,hhmmss,trace_id,travel_time,delay,geometry
0,1,1368,1369,0,56.51,0000:00,0,0,0,"LINESTRING (-2.2630822 53.4842625, -2.2630836 ..."
1,1,1369,1498,0,168.37,0000:00,0,0,0,"LINESTRING (-2.2630836 53.4847707, -2.2633385 ..."
2,1,1498,250,0,299.87,0000:00,0,0,0,"LINESTRING (-2.2647708 53.4847242, -2.2647338 ..."
3,1,250,251,0,444.01,0000:00,0,0,0,"LINESTRING (-2.2647338 53.4859066, -2.2646622 ..."
4,1,251,523,0,486.94,0000:00,3,0,0,"LINESTRING (-2.2643488 53.4871558, -2.2641002 ..."
...,...,...,...,...,...,...,...,...,...,...
1064,179,838,1419,0,166.32,0000:00,1,0,0,"LINESTRING (-2.230529 53.4836417, -2.2303505 5..."
1065,180,574,645,0,145.86,0000:00,0,0,0,"LINESTRING (-2.2227737 53.4816081, -2.222885 5..."
1066,180,645,843,0,157.07,0000:00,1,0,0,"LINESTRING (-2.2244469 53.4807597, -2.2245739 ..."
1067,181,296,297,0,36.63,0000:00,1,0,0,"LINESTRING (-2.2522538 53.4815529, -2.2524769 ..."


In [25]:
measurement_utd

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,timestamp,volume,travel_time,speed,reference_speed,density,queue,notes
0,89398.0,Silk Street,1.0,100000001.0,100000002.0,2017-09-29T23:10,24.0,NaN,40.000000,NaN,NaN,NaN,NaN
1,89398.0,Silk Street,1.0,100000001.0,100000002.0,2017-09-29T23:15,96.0,NaN,41.220339,NaN,NaN,NaN,NaN
2,89398.0,Silk Street,1.0,100000001.0,100000002.0,2017-09-29T23:20,120.0,NaN,38.000000,NaN,NaN,NaN,NaN
3,89398.0,Silk Street,1.0,100000001.0,100000002.0,2017-09-29T23:25,72.0,NaN,35.945946,NaN,NaN,NaN,NaN
4,89398.0,Silk Street,1.0,100000001.0,100000002.0,2017-09-29T23:30,96.0,NaN,35.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6259,90338.0,Leftbank,4.0,100000941.0,100000942.0,2017-09-29T23:25,162.0,NaN,37.000000,NaN,NaN,NaN,NaN
6260,90338.0,Leftbank,4.0,100000941.0,100000942.0,2017-09-29T23:30,144.0,NaN,37.000000,NaN,NaN,NaN,NaN
6261,90338.0,Leftbank,4.0,100000941.0,100000942.0,2017-09-29T23:35,210.0,NaN,30.712980,NaN,NaN,NaN,NaN
6262,90338.0,Leftbank,4.0,100000941.0,100000942.0,2017-09-29T23:40,264.0,NaN,30.000000,NaN,NaN,NaN,NaN


In [26]:
trace

,corridor_id,agent_id,date,utd,trace_id,hh,mm,ss,y_coord,x_coord
0,Silk Street,1,NaN,89398,0,NaN,None,NaN,53.485628,-2.264742
1,Silk Street,1,NaN,89399,1,NaN,None,NaN,53.486187,-2.264694
2,Silk Street,1,NaN,89400,2,NaN,None,NaN,53.486820,-2.264653
3,Silk Street,1,NaN,89401,3,NaN,None,NaN,53.487190,-2.264374
4,Silk Street,1,NaN,89402,4,NaN,None,NaN,53.487496,-2.263860
...,...,...,...,...,...,...,...,...,...,...
937,Leftbank,181,NaN,90335,0,NaN,None,NaN,53.481483,-2.252004
938,Leftbank,181,NaN,90336,1,NaN,None,NaN,53.481538,-2.252378
939,Leftbank,181,NaN,90337,2,NaN,None,NaN,53.481669,-2.252817
940,Leftbank,181,NaN,90338,3,NaN,None,NaN,53.481823,-2.253200


In [27]:
link_road

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway
0,Cobourg Street,0.0,796758.0,114.0,113.0,NaN,84.376463,NaN,30.0,NaN,residential,6.0,"LINESTRING (-2.2331957 53.4759624, -2.2332846 ...",-1,0.0
1,South Pump Street,1.0,797061.0,2184.0,121.0,NaN,36.928887,NaN,30.0,NaN,residential,6.0,"LINESTRING (-2.2320231 53.4765284, -2.2318448 ...",-1,1.0
2,South Pump Street,2.0,797061.0,121.0,2184.0,NaN,36.928887,NaN,30.0,NaN,residential,6.0,"LINESTRING (-2.2318448 53.4762137, -2.2320231 ...",-1,1.0
3,Echo Street,3.0,797062.0,122.0,2192.0,NaN,3.911961,NaN,NaN,NaN,unclassified,11.0,"LINESTRING (-2.2316462 53.4757191, -2.2316626 ...",-1,0.0
4,Granby Row,4.0,797065.0,100.0,1654.0,NaN,62.208520,NaN,20.0,NaN,residential,6.0,"LINESTRING (-2.2376883 53.4748264, -2.2368362 ...",-1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,auto,NaN
6135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,auto,NaN


In [28]:
measurement_osm = pd.DataFrame()
measurement_osm_sub = pd.DataFrame()

measurement_osm['link_id'] = None
measurement_osm['from_node_id'] = None
measurement_osm['to_node_id'] = None
measurement_osm['lanes'] = None
measurement_osm['length'] = None
measurement_osm['time_period'] = None
measurement_osm['date'] = None
measurement_osm['geometry'] = None
measurement_osm['volume'] = None
measurement_osm['speed'] = None


link_road_dict = {}
gp = link_road.groupby(['from_node_id','to_node_id'])
for key, form in gp:
    link_road_dict[key] = {
        'link_id':form['link_id'].values[0],
        'from_node_id':form['from_node_id'].values[0],
        'to_node_id':form['to_node_id'].values[0],
        'lanes':form['lanes'].values[0],
        'length':form['length'].values[0],
        'geometry':form['geometry'].values[0],
        }


trace_dict = {}
gp = trace.groupby(['agent_id','trace_id'])
for key, form in gp:
    trace_dict[key] = {
        'utd':form['utd'].values[0]
        }


measurement_utd_dict = {}
gp = measurement_utd.groupby('name')
for key, form in gp:
    measurement_utd_dict[key] = {
        'timestamp':form['timestamp'].tolist(),
        'volume':form['volume'].tolist(),
        'speed':form['speed'].tolist()
        }


for i in mapping.index:
    link_road_selected = link_road_dict[(mapping.loc[i,'from_node_id'],mapping.loc[i,'to_node_id'])] #find osm link from mapping 
    trace_selected = trace_dict[(mapping.loc[i,'agent_id'],mapping.loc[i,'trace_id'])] #find utd link from mapping
    measurement_utd_dict_selected = measurement_utd_dict[trace_selected['utd']] #find utd performance from utd link

    for j in range(0,len(measurement_utd_dict_selected['timestamp']),3):
        try:
            measurement_osm_sub = measurement_osm_sub.append({'link_id': link_road_selected['link_id'],\
                                            'from_node_id': link_road_selected['from_node_id'],\
                                            'to_node_id': link_road_selected['to_node_id'],\
                                            'lanes': link_road_selected['lanes'],\
                                            'length': round(link_road_selected['length']),\
                                            'time_period': measurement_utd_dict_selected['timestamp'][j][-5:-3] + \
                                                            measurement_utd_dict_selected['timestamp'][j][-2:] + '_'+ \
                                                            measurement_utd_dict_selected['timestamp'][j+3][-5:-3] + \
                                                            measurement_utd_dict_selected['timestamp'][j+3][-2:],\
                                            'date': measurement_utd_dict_selected['timestamp'][j][:10],\
                                            'geometry': link_road_selected['geometry'],\
                                            'volume': np.sum(measurement_utd_dict_selected['volume'][j:j+3]),\
                                            'speed': round(np.mean(measurement_utd_dict_selected['speed'][j:j+3]))}, ignore_index=True)

        except:
            break
            # measurement_osm_sub = measurement_osm_sub.append({'link_id': link_road_selected['link_id'],\
            #                                 'from_node_id': link_road_selected['from_node_id'],\
            #                                 'to_node_id': link_road_selected['to_node_id'],\
            #                                 'lanes': link_road_selected['lanes'],\
            #                                 'length': round(link_road_selected['length']),\
            #                                 'time_period': None,\
            #                                 'date': None,\
            #                                 'geometry': link_road_selected['geometry'],\
            #                                 'volume': None,\
            #                                 'speed': None}, ignore_index=True)


    measurement_osm = measurement_osm.append(measurement_osm_sub)
    measurement_osm_sub = pd.DataFrame()
measurement_osm.to_csv('measurement_osm.csv')
print("measurement_osm.csv is generated!")

measurement_osm.csv is generated!


In [29]:
measurement_osm

,link_id,from_node_id,to_node_id,lanes,length,time_period,date,geometry,volume,speed
0,2460.0,1368.0,1369.0,NaN,57.0,2310_2325,2017-09-29,"LINESTRING (-2.2630822 53.4842625, -2.2630836 ...",240.0,40.0
1,2460.0,1368.0,1369.0,NaN,57.0,2325_2340,2017-09-29,"LINESTRING (-2.2630822 53.4842625, -2.2630836 ...",228.0,36.0
0,2256.0,1369.0,1498.0,NaN,112.0,2310_2325,2017-09-29,"LINESTRING (-2.2630836 53.4847707, -2.2633385 ...",240.0,40.0
1,2256.0,1369.0,1498.0,NaN,112.0,2325_2340,2017-09-29,"LINESTRING (-2.2630836 53.4847707, -2.2633385 ...",228.0,36.0
0,2050.0,1498.0,250.0,NaN,131.0,2310_2325,2017-09-29,"LINESTRING (-2.2647708 53.4847242, -2.2647338 ...",240.0,40.0
...,...,...,...,...,...,...,...,...,...,...
1,1879.0,645.0,843.0,NaN,11.0,2325_2340,2017-09-29,"LINESTRING (-2.2244469 53.4807597, -2.2245739 ...",288.0,33.0
0,2303.0,296.0,297.0,NaN,37.0,2310_2325,2017-09-29,"LINESTRING (-2.2522538 53.4815529, -2.2524769 ...",582.0,36.0
1,2303.0,296.0,297.0,NaN,37.0,2325_2340,2017-09-29,"LINESTRING (-2.2522538 53.4815529, -2.2524769 ...",516.0,35.0
0,2231.0,297.0,1276.0,NaN,39.0,2310_2325,2017-09-29,"LINESTRING (-2.2527645 53.4816785, -2.2529222 ...",582.0,36.0


In [30]:
print( __name__)
print(os.path.dirname(os.path.realpath('__file__')))
print(os.path.realpath('__file__'))
print(os.getcwd())

__main__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/__file__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
